<a href="https://colab.research.google.com/github/AbiemwenseMaureenOshobugie/Analytics-Vidhya/blob/main/intraday_overnight_effects_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Price Changes Breakdown
* Compares **net price movement** across 10 major global indices from 2008 to 2025.
* Quantifies the **net movement and its breakdown** into intraday and overnight components.



In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import warnings
warnings.filterwarnings('ignore')
from IPython.display import Javascript



Magnitide-based gap

This method calculates gap-up/down move magnitudes based on continuos up/down trends until a 10% reversal, either draw-down or draw-up, is hit. Then sums up the total magnitude gap moves.

In [ ]:

def compute_gap_magnitudes(prices, threshold=0.10):
    log_prices = np.log(prices)
    direction = None
    peak = trough = log_prices.iloc[0]
    gap_up_magnitude = 0
    gap_down_magnitude = 0
    move_start = log_prices.iloc[0]

    for i in try range(1, len(log_prices)):
        price = log_prices.iloc[i]

        if direction is None:
            if price > log_prices.iloc[i - 1]:
                direction = 'up'
                peak = price
                move_start = log_prices.iloc[i - 1]
            elif price < log_prices.iloc[i - 1]:
                direction = 'down'
                trough = price
                move_start = log_prices.iloc[i - 1]
            continue

        if direction == 'up':
            if price > peak:
                peak = price
            drawdown = (peak - price)
            if drawdown >= threshold:
                gap_up_magnitude += (peak - move_start)
                direction = None
        elif direction == 'down':
            if price < trough:
                trough = price
            drawup = (price - trough)
            if drawup >= threshold:
                gap_down_magnitude += (move_start - trough)
                direction = None

    return gap_up_magnitude, gap_down_magnitude


Changes overnight and intraday

In [ ]:

indices = ['^GSPC', '^NDX', '^DJI', '^FTSE', '^FCHI', '^GDAXI', '^NSEI', '^BSESN', '^HSI', '^N225']
effects = pd.DataFrame(columns=[
    'index', 'intraday_change', 'overnight_change',
    'net_change', 'net_pct_change',
    'gap_up_magnitude', 'gap_down_magnitude'
])

for i, idx in enumerate(indices):
    start = '2008-01-01'
    end = '2025-04-30'
    data = yf.download(idx, start=start, end=end, auto_adjust=False, progress=False)
    if data.empty:
        continue
    data.columns = data.columns.get_level_values(0)

    # Calculate intraday and overnight returns
    data['intraday_change'] = data['Close'] - data['Open']
    data['overnight_change'] = data['Open'] - data['Close'].shift(1)

    # Magnitude-based gap moves
    gap_up_magnitude, gap_down_magnitude = compute_gap_magnitudes(data['Close'])

    # Net change and net % change
    if len(data) > 0:
        initial_open = data['Open'].iloc[0]
        final_close = data['Close'].iloc[-1]
        net_change = final_close - initial_open
        net_pct_change = (net_change / initial_open) * 100
    else:
        net_change = 0
        net_pct_change = 0

    effects.loc[i] = [
        idx,
        data['intraday_change'].sum(),
        data['overnight_change'].sum(),
        net_change,
        net_pct_change,
        gap_up_magnitude,
        gap_down_magnitude
    ]

effects = effects.round(2)
print("\n", effects.to_string(index=False))

# Summary
total_intraday = effects['intraday_change'].astype(float).sum()
total_overnight = effects['overnight_change'].astype(float).sum()
total_net_change = effects['net_change'].astype(float).sum()

weights = effects['net_change'].abs() / effects['net_change'].abs().sum()
total_net_pct_change = (effects['net_pct_change'] * weights).sum()

intraday_pct = 100 * total_intraday / total_net_change if total_net_change != 0 else 0
overnight_pct = 100 * total_overnight / total_net_change if total_net_change != 0 else 0

total_gap_ups = effects['gap_up_magnitude'].astype(float).sum()
total_gap_downs = effects['gap_down_magnitude'].astype(float).sum()

print("\n=== Summary ===")
print("Total Intraday Change:   ", round(total_intraday, 4))
print("Total Overnight Change:  ", round(total_overnight, 4))
print("Total Net Change:        ", round(total_net_change, 4))

print("\nIntraday % of Net:       ", round(intraday_pct, 2), "%")
print("Overnight % of Net:      ", round(overnight_pct, 2), "%")
print("Total Net % Change:      ", round(total_net_pct_change, 2), "%")

print("\nTotal Gap-Up Magnitude:  ", round(total_gap_ups))
print("Total Gap-Down Magnitude:", round(total_gap_downs))



  index  intraday_change  overnight_change  net_change  net_pct_change  gap_up_magnitude  gap_down_magnitude
 ^GSPC          1711.40           2103.33     3814.73          259.86              3.35                1.75
  ^NDX          6809.68           9362.88    16172.56          775.47              5.43                2.14
  ^DJI         14665.75          11214.66    25880.41          195.15              3.31                1.06
 ^FTSE          1802.30             16.50     1818.80           28.17              1.74                1.98
 ^FCHI         -3629.22           5305.10     1675.88           29.87              3.28                2.75
^GDAXI         -1401.94          14561.83    13159.89          163.56              4.43                2.80
 ^NSEI        -28851.29          46840.09    17988.80          293.13              4.66                1.93
^BSESN       -127161.14         186176.54    59015.40          289.39              3.69                3.11
  ^HSI        -58613.56   

Time-level snapshot across the entire date range

In [ ]:

# Display settings for Google Colab (for adjusting iframe height)
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'''))

# Indices and time periods
indices = ['^GSPC', '^NDX', '^DJI', '^FTSE', '^FCHI', '^GDAXI', '^NSEI', '^BSESN', '^HSI', '^N225']
time_periods = [
    ('2008-01-01', '2010-12-31'),
    ('2011-01-01', '2015-12-31'),
    ('2016-01-01', '2019-12-31'),
    ('2020-01-01', '2022-12-31'),
    ('2023-01-01', '2025-04-30')
]

# Dictionary to store results per period
period_dataframes = {}

# Download all index data first
print("Downloading all index data...")
index_data = {}
for idx in indices:
    index_data[idx] = yf.download(idx, start='2008-01-01', end='2025-04-30', auto_adjust=False, progress=False)
    if isinstance(index_data[idx].columns, pd.MultiIndex):
        index_data[idx].columns = index_data[idx].columns.get_level_values(0)

# Process each time period
for period_start, period_end in time_periods:
    period_key = f"{period_start} to {period_end}"
    period_results = []

    for idx in indices:
        data = index_data[idx].loc[period_start:period_end].copy()

        if data.empty or len(data) < 2:
            continue

        # Price changes
        data['intraday_change'] = data['Close'] - data['Open']
        data['overnight_change'] = data['Open'] - data['Close'].shift(1)

        # Magnitude-based gap moves
        gap_up_magnitude, gap_down_magnitude = compute_gap_magnitudes(data['Close'])

        initial_open = data['Open'].iloc[0]
        final_close = data['Close'].iloc[-1]
        net_change = final_close - initial_open
        net_pct_change = (net_change / initial_open) * 100

        # Store the results for the current period and index
        period_results.append({
            'index': idx,
            'total_days': len(data),
            'intraday_change': data['intraday_change'].sum(),
            'overnight_change': data['overnight_change'].sum(),
            'net_change': net_change,
            'net_pct_change': net_pct_change,
            'gap_up_magnitude ': gap_up_magnitude,
            'gap_down_magnitude ': gap_down_magnitude

        })
    # Round values inside each dictionary in the list
    rounded_results = [
       {k: round(v, 2) if isinstance(v, (int, float)) else v for k, v in result.items()}
       for result in period_results
        ]
    period_dataframes[period_key] = pd.DataFrame(rounded_results)

for period, df in period_dataframes.items():
    print(f"\n==== Period: {period} ====\n")
    print(df.to_string())


<IPython.core.display.Javascript object>


==== Period: 2008-01-01 to 2010-12-31 ====

    index  total_days  intraday_change  overnight_change  net_change  net_pct_change  gap_up_magnitude   gap_down_magnitude 
0   ^GSPC         757           -93.92           -116.41     -210.33          -14.33               0.63                 1.03
1    ^NDX         757           228.71            -96.38      132.33            6.35               1.21                 0.71
2    ^DJI         757          -677.98          -1006.33    -1684.31          -12.70               0.71                 0.52
3   ^FTSE         760          -564.70              7.70     -557.00           -8.63               0.77                 0.95
4   ^FCHI         768         -3374.84           1615.62    -1759.22          -31.36               0.77                 1.30
5  ^GDAXI         764         -2231.28           1099.50    -1131.78          -14.07               0.92                 1.02
6   ^NSEI         736         -1645.09           1642.84       -2.25           -

Region-level snapshot across the entire date range


In [ ]:

indices = {
    'US': ['^GSPC', '^NDX', '^DJI'],  # S&P 500, NASDAQ, Dow Jones
    'Europe': ['^FTSE', '^FCHI', '^GDAXI'],  # FTSE 100, CAC 40, DAX
    'Asia': ['^NSEI', '^BSESN', '^HSI', '^N225']  # Nifty 50, Sensex, Hang Seng, Nikkei 225
}

region_summary_full = {}

for region, region_indices in indices.items():
    region_results = []
    for ridx in region_indices:
        data = index_data[ridx].copy()

        if data.empty or len(data) < 2:
            continue

        data['intraday_change'] = data['Close'] - data['Open']
        data['overnight_change'] = data['Open'] - data['Close'].shift(1)

        gap_up_magnitude, gap_down_magnitude = compute_gap_magnitudes(data['Close'])

        initial_open = data['Open'].iloc[0]
        final_close = data['Close'].iloc[-1]
        net_change = final_close - initial_open
        net_pct_change = (net_change / initial_open) * 100

        region_results.append({
            'region': region,
            'index': ridx,
            'total_days': len(data),
            'intraday_change': data['intraday_change'].sum(),
            'overnight_change': data['overnight_change'].sum(),
            'net_change': net_change,
            'net_pct_change': net_pct_change,
            'gap_up_magnitude': gap_up_magnitude,
            'gap_down_magnitude': gap_down_magnitude
        })

    # Round the values in region_results
    rounded_results = [
        {k: round(v, 2) if isinstance(v, (int, float)) else v for k, v in row.items()}
        for row in region_results
    ]

    region_summary_full[region] = pd.DataFrame(rounded_results)
    print(f"\n==== Region Snapshot: {region} ====\n")
    print(region_summary_full[region].to_string())



==== Region Snapshot: US ====

  region  index  total_days  intraday_change  overnight_change  net_change  net_pct_change  gap_up_magnitude  gap_down_magnitude
0     US  ^GSPC        4353          1654.81           2053.57     3708.38          252.62              3.35                1.75
1     US   ^NDX        4353          6634.64           9127.79    15762.43          755.80              5.43                2.14
2     US   ^DJI        4353         14148.34          10978.43    25126.77          189.47              3.31                1.06

==== Region Snapshot: Europe ====

   region   index  total_days  intraday_change  overnight_change  net_change  net_pct_change  gap_up_magnitude  gap_down_magnitude
0  Europe   ^FTSE        4369          1802.30             16.50     1818.80           28.17              1.74                1.98
1  Europe   ^FCHI        4424         -3629.22           5305.10     1675.88           29.87              3.28                2.75
2  Europe  ^GDAXI      